# <span style="color:#0b486b">  FIT3181: Deep Learning (2022)</span>
***
*CE/Lecturer (Clayton):*  **Dr Trung Le** | trunglm@monash.edu <br/>
*Lecturer (Malaysia):*  **Dr Lim Chern Hong** | lim.chernhong@monash.edu <br/>  <br/>
*Tutor:*  **Mr Thanh Nguyen** \[Thanh.Nguyen4@monash.edu \] |**Mr Tuan Nguyen**  \[tuan.ng@monash.edu \] |**Mr Anh Bui** \[tuananh.bui@monash.edu\] | **Dr Binh Nguyen** \[binh.nguyen1@monash.edu \] | **Mr Md Mohaimenuzzaman** \[md.mohaimen@monash.edu \] |**Mr James Tong** \[james.tong1@monash.edu \]
<br/> <br/>
Faculty of Information Technology, Monash University, Australia
***

The tutorial will be using `sklearn` to load the dataset. 

In [11]:
import os 
import numpy as np
from sklearn.datasets import load_svmlight_file 

In [12]:
data_file_name = "letter_scale.libsvm"
data_file = os.path.abspath("./Data/" + data_file_name)
X_data, y_data = load_svmlight_file(data_file)
X_data = X_data.toarray() 
y_data = y_data.reshape(y_data.shape[0], -1)

# Getting the shape of data x and y
print(f"X data shape:{X_data.shape}")
print(f"y data shape:{y_data.shape}")
print(f"Number of unique classes: {len(np.unique(y_data))}")
print(f"Checkout label classes: {np.unique(y_data)}")

X data shape:(15000, 16)
y data shape:(15000, 1)
Number of unique classes: 26
Checkout label classes: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26.]


In [13]:
# What does data x and y have?
print(f"X Sample data:{X_data[0]}") # Condensed data from dataset
print(f"y Sample data:{y_data[0]}") # Label from dataset

X Sample data:[-0.733333  -0.466667  -0.466667  -0.6       -0.733333  -0.0666667
  0.0666667 -0.733333   0.2        0.466667  -0.0666667 -0.0666667
 -0.866667   0.0666667 -0.333333  -0.2      ]
y Sample data:[26.]


Using `sklearn` to yet again split the dataset into train, validation and test sets. 

In [14]:
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing

def train_valid_test_split(data, target, train_size, test_size):
    validation_size = 1 - (train_size + test_size)
    
    # Performing the normal split, then only split the training set again as validation set
    X1, X_test, y1, y_test = train_test_split(data, target, test_size=test_size, random_state=42)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X1, y1, test_size=float(validation_size)/(validation_size + train_size))
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [15]:
le = preprocessing.LabelEncoder()
le.fit(y_data.ravel())
y_data = le.transform(y_data.ravel())
print(f"how y_data looks like: {y_data}")


how y_data looks like: [25 15 18 ...  0 11 21]


In [16]:
X_train, X_valid, X_test, y_train, y_valid, y_test = train_valid_test_split(X_data,y_data, train_size=0.8, test_size=0.1)
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)
y_valid = y_valid.reshape(-1)

print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(12000, 16) (1500, 16) (1500, 16)
(12000,) (1500,) (1500,)


## Building the model

In [20]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.models import Sequential 

# Using `tf.keras.Model as inheritance`
class MyDNN(tf.keras.Model):
    def __init__(self, n_classes= 26):
        super(MyDNN, self).__init__()
        self.n_classes = n_classes
        self.dense1 = tf.keras.layers.Dense(units=10, activation= 'relu')
        self.dense2 = tf.keras.layers.Dense(units=20, activation= 'relu')
        self.dense3 = tf.keras.layers.Dense(units=15, activation= 'relu')
        self.dense4 = tf.keras.layers.Dense(units=self.n_classes, activation= 'softmax')
    
    def call(self,X): #X is the input, method call specifies how to compute the output from the input X
        h = self.dense1(X)
        h = self.dense2(h)
        h = self.dense3(h)
        h = self.dense4(h)
        return h
dnn_model = MyDNN(n_classes= 26)
dnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn_model.fit(x=X_train, y=y_train, batch_size=32, 
                        epochs=20, 
                        validation_data=(X_valid, y_valid))
         
            
        

Epoch 1/20
375/375 [==============================] - 1s 877us/step - loss: 2.7968 - accuracy: 0.1863 - val_loss: 2.2122 - val_accuracy: 0.3433
Epoch 2/20
375/375 [==============================] - 0s 642us/step - loss: 1.8395 - accuracy: 0.4452 - val_loss: 1.7003 - val_accuracy: 0.5087
Epoch 3/20
375/375 [==============================] - 0s 639us/step - loss: 1.5493 - accuracy: 0.5297 - val_loss: 1.5596 - val_accuracy: 0.5393
Epoch 4/20
375/375 [==============================] - 0s 644us/step - loss: 1.4327 - accuracy: 0.5682 - val_loss: 1.4621 - val_accuracy: 0.5673
Epoch 5/20
375/375 [==============================] - 0s 703us/step - loss: 1.3478 - accuracy: 0.5988 - val_loss: 1.3820 - val_accuracy: 0.5907
Epoch 6/20
375/375 [==============================] - 0s 650us/step - loss: 1.2738 - accuracy: 0.6235 - val_loss: 1.3186 - val_accuracy: 0.6060
Epoch 7/20
375/375 [==============================] - 0s 647us/step - loss: 1.2095 - accuracy: 0.6413 - val_loss: 1.2639 - val_accuracy:

In [19]:
n_epochs =20
batch_size = 32
for epoch in range(n_epochs):
    for idx_start in range(0, X_train.shape[0], batch_size):
        idx_end = min(X_train.shape[0], idx_start + batch_size)
        X_batch, y_batch = X_train[idx_start:idx_end], y_train[idx_start:idx_end]
        train_loss_batch = dnn_model.train_on_batch(X_batch, y_batch)  #return the batch loss
        
    train_loss, train_acc = dnn_model.evaluate(x= X_train, y= y_train, batch_size= 64, verbose= 0)
    valid_loss, valid_acc = dnn_model.evaluate(x= X_valid, y= y_valid, batch_size= 64, verbose= 0)
    print('Epoch {}: train acc={:.4f}, train loss={:.4f} | valid acc={:.4f}, valid loss= {:.4f}'.format(epoch +1, 
                                                                                                        train_acc, 
                                                                                                        train_loss, 
                                                                                                        valid_acc, 
                                                                                                        valid_loss))

Epoch 1: train acc=0.7338, train loss=0.9215 | valid acc=0.7113, valid loss= 0.9923
Epoch 2: train acc=0.7393, train loss=0.8966 | valid acc=0.7207, valid loss= 0.9647
Epoch 3: train acc=0.7456, train loss=0.8734 | valid acc=0.7260, valid loss= 0.9404
Epoch 4: train acc=0.7497, train loss=0.8537 | valid acc=0.7300, valid loss= 0.9197
Epoch 5: train acc=0.7548, train loss=0.8361 | valid acc=0.7387, valid loss= 0.9019
Epoch 6: train acc=0.7575, train loss=0.8202 | valid acc=0.7393, valid loss= 0.8860
Epoch 7: train acc=0.7604, train loss=0.8068 | valid acc=0.7400, valid loss= 0.8723
Epoch 8: train acc=0.7638, train loss=0.7942 | valid acc=0.7420, valid loss= 0.8595
Epoch 9: train acc=0.7664, train loss=0.7825 | valid acc=0.7420, valid loss= 0.8476
Epoch 10: train acc=0.7682, train loss=0.7728 | valid acc=0.7467, valid loss= 0.8372
Epoch 11: train acc=0.7709, train loss=0.7627 | valid acc=0.7493, valid loss= 0.8269
Epoch 12: train acc=0.7727, train loss=0.7537 | valid acc=0.7493, valid lo